In [ ]:
import os
import re
import time
import csv
import json
import random
import requests
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By

In [ ]:
pip install selenium==3.141.0

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


### Set up the chrome driver by passing some parameters

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = False
browser = webdriver.Chrome(executable_path = 'C:/Users/shrey/Downloads/chromedriver_win32/chromedriver.exe',
                          options = chrome_options)
url_head = 'https://www.estimize.com/users/sign_in'
browser.get(url_head)
time.sleep(random.uniform(1,5))

### Start of getting the Estimize data

#### filling in sign_in forms by identifying the username and password cells

In [ ]:
def sign_in():
    url = 'https://www.estimize.com/users/sign_in'
    browser.get(url)


    
    email_field = browser.find_element_by_id('user_login')
    password_field = browser.find_element_by_id('user_password')
    submit_btn = browser.find_element_by_xpath('//input[@type="submit"]')




    username = 'shubmukh@gmail.com'
    password = 'Shubhomoy_2010'
    
    

    email_field.send_keys(username)
    password_field.send_keys(password)
    submit_btn.click()

    time.sleep(1)

In [ ]:
sign_in()

#### Test on Estimize website for getting the data

In [ ]:
# url = 'https://www.glassdoor.com/Overview/Working-at-ADC-Telecommunications-EI_IE1075.11,33.htm'

tickers = ['A', 'AA', 'AACH', 'AAL', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAWW', 'AAXN','ABBV', 'ABC', 'ABEO', 'ABG','ABM','ABMD', 'ABT', 
           'ACA', 'ACAD', 'ACB', 'ACCO', 'LULU', 'ACET', 'ACHC', 'ACIA', 'ACIW', 'ACLS','ACM','ACN','ACOR','ACRX', 'ACTG', 'ADAP',
           'ADBE', 'ADI','ADM','ADMS', 'ADP', 'ADS', 'ADSK', 'ADSW', 'ADTN',  'ADUS', 'AEE', 'AEGN', 'AEIS', 'AEL', 'AEM', 'AEO']

quarters = ['fq1-2020', 'fq2-2020', 'fq3-2020', 'fq4-2020']



### Extract data for Company table 

In [ ]:
def scrape_overview(browser):
   
    overviewAll = {}

    try:
        for i in range(len(tickers)):
            for j in range(len(quarters)):
#             url = 'https://www.estimize.com/'+tickers[i]+'/fq3-2021?metric_name=eps&chart=historical'
                url = 'https://www.estimize.com/'+tickers[i]+ '/' + quarters[j]+ '?metric_name=eps&chart=historical'
    #     https://www.estimize.com/a/fq3-2021?metric_name=eps&chart=historical
                browser.get(url)
                time.sleep(1)
                overview = {}
                overview['quarter'] = quarters[j]
                overview['Ticker'] = browser.find_element_by_class_name('release-header-information-title').text
                overview['Company Name'] = browser.find_element_by_class_name('release-header-information-description').text
                sectors = browser.find_element_by_class_name('release-header-information-breadcrumb')
                overview['Sectors'] = sectors.text.split('›')[0]
                overview['Industries'] = sectors.text.split('›')[1]
                numbers = browser.find_elements_by_class_name('summary-sub-title')
                overview['Number of Followers'] = numbers[0].text
                overview['Number of Analysts'] = numbers[1].text

                overviewAll[len(overviewAll)]=overview
    except Exception:
        pass
       
    return overviewAll

### Create Company CSV/JSON File

In [ ]:
company_info = scrape_overview(browser)

with open('Company.json', 'w') as outfile:
    json.dump(company_info, outfile, indent=4)
    
df=pd.DataFrame(company_info)
df_trans = df.T
df_trans.to_csv('Company.csv')

### Extract data for EPS table

In [ ]:
def scrape_eps(browser):
    eps = {}
#     tickers1 = ['A', 'AA' ]
#     try:
    for i in range(len(tickers)):
        for j in range(len(quarters)):
            url = 'https://www.estimize.com/'+tickers[i]+ '/' + quarters[j] + '?metric_name=eps&chart=historical'
            browser.get(url)
            time.sleep(1)
            analysts_generic = browser.find_elements_by_class_name('estimates-tbl-consensus-column')
            values_generic = browser.find_elements_by_class_name('estimates-tbl-consensus-eps')

            for generic_count in range(len(analysts_generic)):
                eps_generic = {}
                eps_generic['quarter'] = quarters[j]
                eps_generic['Ticker'] = browser.find_element_by_class_name('release-header-information-title').text
                eps_generic['Company Name'] = browser.find_element_by_class_name('release-header-information-description').text
                eps_generic['Name'] = analysts_generic[generic_count].text.split('\n')[0]
                eps_generic['Type'] = "Generic"
                eps_generic['Estimated Value'] = values_generic[generic_count].text
                eps [len(eps)] = eps_generic

            analysts = browser.find_elements_by_class_name('username')
            analysts_values = browser.find_elements_by_class_name('estimates-tbl-eps')

            for analyst_count in range(len(analysts)):
                eps_Analyst = {}
                eps_Analyst['quarter'] = quarters[j]
                eps_Analyst['Ticker'] = browser.find_element_by_class_name('release-header-information-title').text
                eps_Analyst['Company Name'] = browser.find_element_by_class_name('release-header-information-description').text
                eps_Analyst['Name'] = analysts[analyst_count].text
                eps_Analyst['Type'] = "Analyst"
                eps_Analyst['Estimated Value'] = analysts_values[analyst_count].text

                eps [len(eps)] = eps_Analyst

    return eps

### Create EPS information Data/JSON File

In [ ]:
eps_info = scrape_eps(browser)

with open('EPS.json', 'w') as outfile:
    json.dump(eps_info, outfile, indent=4)

df=pd.DataFrame(eps_info)
df_trans = df.T
df_trans.to_csv('EPS.csv')

### Remove Duplicates for Analyst URL List

In [ ]:
def Remove_dup(duplicate):
    final_list = []
    
    for num in range(len(duplicate)):
        if duplicate[num] not in final_list:
            final_list.append(duplicate[num])
    return final_list

### Extract All Analyst URLs from the Ticker Pages

In [ ]:
def scrape_analysts_list(browser):
    
    ana_list = {}
    for i in range(len(tickers)):
        for j in range(len(quarters)):
            url = 'https://www.estimize.com/'+tickers[i]+ '/' + quarters[j] + '?metric_name=eps&chart=historical'
            browser.get(url)
            time.sleep(1)

            #analysts = browser.find_elements_by_class_name('username')

    # Get the list of all anchors for a specific Ticker
            lnks=browser.find_elements_by_tag_name("a")

            # traverse list

            for lnk in lnks:
               # get_attribute() to get all href
                if (lnk.get_attribute('href') != None):
                    if ( '/users/' in lnk.get_attribute('href')):
                        if ('sign_out' not in lnk.get_attribute('href')):
                            ana_list[len(ana_list)] = lnk.get_attribute('href')

    ana_list_final = Remove_dup(ana_list)
    print (len(ana_list_final))

    return ana_list_final


### Extract data for Analyst table

In [ ]:
def scrape_analysts_info(browser):

    analysts_url = scrape_analysts_list(browser)
    analyst_Profile = ['Error rate', 'Accuracy Percentile', 'Points', 'Points/Estimate', 'Stocks', 'Pending']
    analysts_Overview_final = {}
    for i in range(len(analysts_url)):
        url = analysts_url[i]
        browser.get(url)
        time.sleep(1)
#         print(url)
        analyst_Overview = {}
        try:
            analyst_Overview['Name'] = browser.find_element_by_class_name('profile-display-name').text
            analyst_Overview['User ID'] = browser.find_element_by_class_name('profile-username').text
            # bio = browser.find_elements_by_class_name('profile-bio-categorizations')
            analyst_Overview['roles'] = browser.find_element_by_class_name('profile-bio-categorizations').text
            date = browser.find_element_by_class_name('profile-activity-stats').text.split (' ')
            analyst_Overview['Join Date'] = date[2] + ' ' + date[3]
            analyst_Overview['Analyst Confidence Score'] = browser.find_element_by_class_name('value').text

            profile_stat = browser.find_elements_by_class_name('profile-stat')
            for i in range (len(profile_stat)):
                analyst_Overview[analyst_Profile[i]] = profile_stat[i].text
           
            analysts_Overview_final[len(analysts_Overview_final)] = analyst_Overview
        except Exception:
            pass
    
    return analysts_Overview_final


### Create Analyst Data/JSON File

In [ ]:
analysts_info = scrape_analysts_info(browser)

with open('Analyst.json', 'w') as outfile:
    json.dump(analysts_info, outfile, indent=4)

df=pd.DataFrame(analysts_info)
df_trans = df.T
df_trans.to_csv('Analyst.csv')

799
